In [1]:
import requests
from bs4 import BeautifulSoup as BS
import re
import pandas as pd

In [2]:
URL = 'https://countrymusichalloffame.org/hall-of-fame/members/'

response = requests.get(URL)

In [3]:
response.status_code

200

In [4]:
soup = BS(response.text)
type(soup)

bs4.BeautifulSoup

## 1. Start by using either the inspector or by viewing the page source. Can you identify a tag that might be helpful for finding the names of all inductees? Make use of this to create a list containing just the names of each inductee.

In [5]:
# https://stackoverflow.com/questions/5041008/how-to-find-elements-by-class
cards = soup.find_all("div", {"class": "vertical-card_content"})

In [6]:
print(cards)

[<div class="vertical-card_content">
<div class="vertical-card_content--title">
<h3>Roy Acuff</h3>
</div>
<div class="vertical-card_content--copy">
<p>
                    Inducted 1962                  </p>
</div>
<div class="fbyf-btn-wrapper inline-text">
<a href="https://countrymusichalloffame.org/artist/roy-acuff/">
<span>Learn More</span>
<svg height="15px" version="1.1" viewbox="0 0 15 15" width="15px" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
<g fill="none" fill-rule="evenodd" id="Symbols" stroke="none" stroke-width="1">
<g id="Icon/CMHOF/arrow-right">
<rect height="15" id="Rectangle" width="15" x="0" y="0"></rect>
<polygon fill="#F0B23A" id="Fill-1" points="8.59732882 13.03125 8.59732882 9.79762459 2 9.79762459 2 4.25168919 8.59732882 4.25168919 8.59732882 1 13.96875 7.01562616"></polygon>
</g>
</g>
</svg>
</a>
</div>
</div>, <div class="vertical-card_content">
<div class="vertical-card_content--title">
<h3>Alabama</h3>
</div>
<div class="ve

In [7]:
#https://www.geeksforgeeks.org/difference-between-find-and-find_all-in-beautifulsoup-python/
#https://www.adamsmith.haus/python/answers/how-to-build-a-pandas-dataframe-with-a-for-loop-in-python
inductees = []
for card in cards:
    inductees.append(str(card.find('h3')))

# inductees = soup.findAll('h3')
# print(type(inductees))
# inductees

In [8]:
inductees

['<h3>Roy Acuff</h3>',
 '<h3>Alabama</h3>',
 '<h3>Bill Anderson</h3>',
 '<h3>Eddy Arnold</h3>',
 '<h3>Chet Atkins</h3>',
 '<h3>Gene Autry</h3>',
 '<h3>DeFord Bailey</h3>',
 '<h3>Bobby Bare</h3>',
 '<h3>Eddie Bayers</h3>']

In [9]:
#https://stackoverflow.com/questions/1249388/removing-all-non-numeric-characters-from-string-in-python
#https://www.codegrepper.com/code-examples/html/regex+to+remove+html+tags+python
inductees = [re.sub('<.*?>', '', x) for x in inductees]
inductees

['Roy Acuff',
 'Alabama',
 'Bill Anderson',
 'Eddy Arnold',
 'Chet Atkins',
 'Gene Autry',
 'DeFord Bailey',
 'Bobby Bare',
 'Eddie Bayers']

In [10]:
# the error I got when I ran this makes me think the above ResultSet is already a list
# ("You're probably treating a list of elements like a single element.")
# inductees = inductees.to_list()

In [11]:
# # I'm not sure this is necessary ?

# # https://www.geeksforgeeks.org/beautifulsoup-scraping-list-from-html/
# # Iterate through all h3 tags
# inductee_list = []
# for data in inductees:
#     # Get text from each tag
#     print(data.text)

## 2. Next, try and find a tag that could be used to find the year that each member was inducted. Extract these into a list. When you do this, be sure to only include the year and not the full text. For example, for Roy Acuff, the list entry should be "1962" and not "Inducted 1962". Double-check that the resulting list has the correct number of elements and is in the same order as your inductees list.

In [12]:
year_inducted = []
for card in cards:
    year_inducted.append(str(card.find('p')))

year_inducted

['<p>\n                    Inducted 1962                  </p>',
 '<p>\n                    Inducted 2005                  </p>',
 '<p>\n                    Inducted 2001                  </p>',
 '<p>\n                    Inducted 1966                  </p>',
 '<p>\n                    Inducted 1973                  </p>',
 '<p>\n                    Inducted 1969                  </p>',
 '<p>\n                    Inducted 2005                  </p>',
 '<p>\n                    Inducted 2013                  </p>',
 '<p>\n                    Inducted 2022                  </p>']

In [13]:
#https://stackoverflow.com/questions/1249388/removing-all-non-numeric-characters-from-string-in-python
year_inducted = [re.sub("[^0-9]", "", x) for x in year_inducted]
year_inducted

['1962', '2005', '2001', '1966', '1973', '1969', '2005', '2013', '2022']

## 3. Take the two lists you created on parts 1 and 2 and convert it into a pandas DataFrame.

In [14]:
#https://cmdlinetips.com/2018/01/how-to-create-pandas-dataframe-from-multiple-lists/
inductees_and_dates = {'artist':inductees, 'year_inducted':year_inducted}
inductees_and_dates = pd.DataFrame(inductees_and_dates)
inductees_and_dates.head(10)

,artist,year_inducted
0,Roy Acuff,1962
1,Alabama,2005
2,Bill Anderson,2001
3,Eddy Arnold,1966
4,Chet Atkins,1973
5,Gene Autry,1969
6,DeFord Bailey,2005
7,Bobby Bare,2013
8,Eddie Bayers,2022


In [15]:
inductees_and_dates.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   artist         9 non-null      object
 1   year_inducted  9 non-null      object
dtypes: object(2)
memory usage: 272.0+ bytes


## 4. Now, you need to take what you created for the first page and apply it across the rest of the pages so that you can scrape all inductees. Notice that when you click the next page button at the bottom of the page that the url changes to "https://countrymusichalloffame.org/hall-of-fame/members/page/2". Check that the code that you wrote for the first page still works for page 2. Once you have verified that your code will still work, write a for loop that will cycle through all 16 pages and build a DataFrame containing all inductees and the year of their induction.

In [16]:
#https://data36.com/scrape-multiple-web-pages-beautiful-soup-tutorial/

#testing
page = 1
while page != 5:
      url = f"https://countrymusichalloffame.org/hall-of-fame/members/page/{page}/"
      print(url)
      page = page + 1

https://countrymusichalloffame.org/hall-of-fame/members/page/1/
https://countrymusichalloffame.org/hall-of-fame/members/page/2/
https://countrymusichalloffame.org/hall-of-fame/members/page/3/
https://countrymusichalloffame.org/hall-of-fame/members/page/4/


In [17]:
#https://data36.com/scrape-multiple-web-pages-beautiful-soup-tutorial/
#https://www.geeksforgeeks.org/how-to-scrape-multiple-pages-of-a-website-using-python/

#initiate data storage
inductees = []
year_inducted = []

page = 1
while page != 19:
    url = f'https://countrymusichalloffame.org/hall-of-fame/members/page/{page}/'
        
    response = requests.get(url)
    html = response.content
    soup = BS(html, 'html.parser')
    cards = soup.find_all("div", {"class": "vertical-card_content"})

    for card in cards:
        inductees.append(str(card.find('h3')))
        year_inducted.append(str(card.find('p')))
    
    page = page + 1

In [18]:
year_inducted

['<p>\n                    Inducted 1962                  </p>',
 '<p>\n                    Inducted 2005                  </p>',
 '<p>\n                    Inducted 2001                  </p>',
 '<p>\n                    Inducted 1966                  </p>',
 '<p>\n                    Inducted 1973                  </p>',
 '<p>\n                    Inducted 1969                  </p>',
 '<p>\n                    Inducted 2005                  </p>',
 '<p>\n                    Inducted 2013                  </p>',
 '<p>\n                    Inducted 2022                  </p>',
 '<p>\n                    Inducted 1999                  </p>',
 '<p>\n                    Inducted 2011                  </p>',
 '<p>\n                    Inducted 2006                  </p>',
 '<p>\n                    Inducted 2019                  </p>',
 '<p>\n                    Inducted 1974                  </p>',
 '<p>\n                    Inducted 1987                  </p>',
 '<p>\n                  

In [19]:
inductees

['<h3>Roy Acuff</h3>',
 '<h3>Alabama</h3>',
 '<h3>Bill Anderson</h3>',
 '<h3>Eddy Arnold</h3>',
 '<h3>Chet Atkins</h3>',
 '<h3>Gene Autry</h3>',
 '<h3>DeFord Bailey</h3>',
 '<h3>Bobby Bare</h3>',
 '<h3>Eddie Bayers</h3>',
 '<h3>Johnny Bond</h3>',
 '<h3>Bobby Braddock</h3>',
 '<h3>Harold Bradley</h3>',
 '<h3>Jerry Bradley</h3>',
 '<h3>Owen Bradley</h3>',
 '<h3>Rod Brasfield</h3>',
 '<h3>Garth Brooks</h3>',
 '<h3>Brooks &amp; Dunn</h3>',
 '<h3>Jim Ed Brown</h3>',
 '<h3>Jim Ed Brown and the Browns</h3>',
 '<h3>Boudleaux and Felice Bryant</h3>',
 '<h3>Glen Campbell</h3>',
 '<h3>Bill Carlisle</h3>',
 '<h3>Carter Family</h3>',
 '<h3>Johnny Cash</h3>',
 '<h3>Ray Charles</h3>',
 '<h3>Roy Clark</h3>',
 '<h3>Cowboy Jack Clement</h3>',
 '<h3>Patsy Cline</h3>',
 '<h3>Hank Cochran</h3>',
 '<h3>Paul Cohen</h3>',
 '<h3>Floyd Cramer</h3>',
 '<h3>Vernon Dalhart</h3>',
 '<h3>Charlie Daniels</h3>',
 '<h3>Jimmie Davis</h3>',
 '<h3>Jimmy Dean</h3>',
 '<h3>Delmore Brothers</h3>',
 '<h3>Jim Denny</h3>',
 '<h

In [20]:
inductees = [re.sub('<.*?>', '', x) for x in inductees]
inductees

['Roy Acuff',
 'Alabama',
 'Bill Anderson',
 'Eddy Arnold',
 'Chet Atkins',
 'Gene Autry',
 'DeFord Bailey',
 'Bobby Bare',
 'Eddie Bayers',
 'Johnny Bond',
 'Bobby Braddock',
 'Harold Bradley',
 'Jerry Bradley',
 'Owen Bradley',
 'Rod Brasfield',
 'Garth Brooks',
 'Brooks &amp; Dunn',
 'Jim Ed Brown',
 'Jim Ed Brown and the Browns',
 'Boudleaux and Felice Bryant',
 'Glen Campbell',
 'Bill Carlisle',
 'Carter Family',
 'Johnny Cash',
 'Ray Charles',
 'Roy Clark',
 'Cowboy Jack Clement',
 'Patsy Cline',
 'Hank Cochran',
 'Paul Cohen',
 'Floyd Cramer',
 'Vernon Dalhart',
 'Charlie Daniels',
 'Jimmie Davis',
 'Jimmy Dean',
 'Delmore Brothers',
 'Jim Denny',
 'Little Jimmy Dickens',
 'Dean Dillon',
 'Pete Drake',
 'Ralph Emery',
 'The Everly Brothers',
 'Flatt And Scruggs',
 'Jim Foglesong',
 'Red Foley',
 'Tennessee Ernie Ford',
 'Fred Foster',
 'Joseph Frank',
 'Lefty Frizzell',
 'Connie B. Gay',
 'Don Gibson',
 'Vince Gill',
 'Johnny Gimble',
 'Merle Haggard',
 'Tom T. Hall',
 'Emmylou 

In [21]:
year_inducted = [re.sub("[^0-9]", "", x) for x in year_inducted]
year_inducted

['1962',
 '2005',
 '2001',
 '1966',
 '1973',
 '1969',
 '2005',
 '2013',
 '2022',
 '1999',
 '2011',
 '2006',
 '2019',
 '1974',
 '1987',
 '2012',
 '2019',
 '2015',
 '2015',
 '1991',
 '2005',
 '2002',
 '1970',
 '1980',
 '2022',
 '2009',
 '2013',
 '1973',
 '2014',
 '1976',
 '2003',
 '1981',
 '2016',
 '1972',
 '2010',
 '2001',
 '1966',
 '1983',
 '2021',
 '2022',
 '2007',
 '2001',
 '1985',
 '2004',
 '1967',
 '1990',
 '2016',
 '1967',
 '1982',
 '1980',
 '2001',
 '2007',
 '2018',
 '1994',
 '2008',
 '2008',
 '1966',
 '2001',
 '1982',
 '1997',
 '2010',
 '2017',
 '2006',
 '2001',
 '1992',
 '1978',
 '2001',
 '1974',
 '2004',
 '2001',
 '1997',
 '1979',
 '2001',
 '1988',
 '1966',
 '2009',
 '2015',
 '2009',
 '2011',
 '1995',
 '2014',
 '1970',
 '1996',
 '1998',
 '2001',
 '1993',
 '2015',
 '1996',
 '1999',
 '1975',
 '1984',
 '2001',
 '2001',
 '1998',
 '1992',
 '1996',
 '2000',
 '2017',
 '1967',
 '1964',
 '2012',
 '1982',
 '1961',
 '2013',
 '1988',
 '1961',
 '1986',
 '1971',
 '2017',
 '2011',
 '2010',
 

In [22]:
inductees_and_dates = {'artist':inductees, 'year_inducted':year_inducted}
inductees_and_dates = pd.DataFrame(inductees_and_dates)
inductees_and_dates.head(150)

,artist,year_inducted
0,Roy Acuff,1962
1,Alabama,2005
2,Bill Anderson,2001
3,Eddy Arnold,1966
4,Chet Atkins,1973
...,...,...
142,Hank Williams Jr.,2021
143,Bob Wills,1968
144,Mac Wiseman,2014
145,Tammy Wynette,1998


## Bonus: If you navigate to Roy Acuff's page, you will see that his date of birth and date of death are listed towards the top of the page, along with his birthplace. Write some code that will extract these three values. Once you get it working for Roy Acuff, figure out how you can automatically extract these values across the whole dataset of artists. In doing this, you'll need to figure out a way to automatically determine the correct urls for each artist. Note also that not every artist will have these three values, so write your code in a way that it can handle cases where these values are missing. Alabama is one such example.

In [23]:
URL = 'https://countrymusichalloffame.org/artist/roy-acuff/'
response = requests.get(URL)
soup = BS(response.text)

# dob = []
# dod = []
# birthplace = []

dob = soup.find_all('p', text=re.compile('(?<=Birth:)\s\w+\s\d+,\s\d+'))
#(r'BIRTH: \w{3,9}?\s\d{1,2}?\s,\s\d{4}?')

In [24]:
print(dob) 

[]


In [25]:
soup.find_all(text=re.compile('BIRTH: \w'))

[]